## Feature Engineering Refinement
Utilise the comprehensive feature analysis from feature_engineering to select optimal feature sets for model training.

In [3]:
from pathlib import Path
import json
import pandas as pd

# load feature importance rankings
processed_dir = Path('data/processed')
importance_df = pd.read_csv('../data/processed/feature_importance_analysis.csv')

# load selected feature sets
with open(('../data/processed/selected_feature_sets.json'), 'r') as f:
    feature_sets = json.load(f)

print("available feature sets:")
for set_name, features in feature_sets.items():
    print(f"  {set_name}: {len(features)} features")

available feature sets:
  minimal_set: 3 features
  recommended_set: 4 features
  comprehensive_set: 4 features
  tier1_features: 3 features
  tier2_features: 1 features
  tier3_features: 0 features
  tier4_features: 22 features


## Data preparation

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
from imblearn.over_sampling import SMOTE
import time

import pandas as pd
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import seaborn as sns

# set random seed for reproducibility
SEED = 2025
np.random.seed(SEED)

def prepare_modelling_data(df: pd.DataFrame, target: str = 'target') -> Tuple:
    """
    prepare data for model training with proper encoding and splitting
    
    args:
        df: preprocessed dataframe
        target: target variable name
        
    returns:
        tuple of (X_train, X_val, X_test, y_train, y_val, y_test, feature_names, encoders)
    """
    df_model_prep = df.copy()
    
    # separate features and target
    if target not in df_model_prep.columns:
        raise ValueError(f"target column '{target}' not found in dataframe")
    
    y = df_model_prep[target]
    X = df_model_prep.drop(columns=[target])
    
    # encode categorical variables
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    encoders = {}
    
    for col in categorical_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        encoders[col] = le
    
    # convert datetime columns to numeric
    datetime_cols = X.select_dtypes(include=['datetime64']).columns.tolist()
    for col in datetime_cols:
        X[f'{col}_year'] = X[col].dt.year
        X[f'{col}_month'] = X[col].dt.month
        X = X.drop(columns=[col])
    
    feature_names = X.columns.tolist()
    
    # Set your upper and lower bounds
    lower_bound = -1e6
    upper_bound = 1e6

    # Apply across all numeric columns
    X = X.clip(lower=lower_bound, upper=upper_bound)
    
    # split data: 70% train, 15% validation, 15% test
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.30, stratify=y, random_state=SEED
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=SEED
    )
    
    print(f"train set: {len(X_train):,} samples")
    print(f"validation set: {len(X_val):,} samples")
    print(f"test set: {len(X_test):,} samples")
    print(f"features: {len(feature_names)}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test, feature_names, encoders

# load data
df_model_processed = pd.read_csv("/Users/chenjing/Desktop/credit-risk-prediction/data/processed/accepted_loans_model_ready.csv")


# prepare data
X_train, X_val, X_test, y_train, y_val, y_test, feature_names, encoders = prepare_modelling_data(
    df_model_processed
)

train set: 211,610 samples
validation set: 45,345 samples
test set: 45,346 samples
features: 26


## Feature Set Comparison Strategy
Systematically compare model performance across different feature sets to identify optimal feature configuration.

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from typing import Dict, List

def compare_feature_sets(X_train: pd.DataFrame, y_train: pd.Series, 
                        feature_sets: Dict[str, List[str]], 
                        cv: int = 5, 
                        random_state: int = 2025) -> pd.DataFrame:
    """
    evaluate model performance across different feature sets using cross-validation
    
    args:
        X_train: training features dataframe
        y_train: training target series
        feature_sets: dictionary mapping set names to feature lists
        cv: number of cross-validation folds (default 5)
        random_state: random seed for reproducibility
        
    returns:
        dataframe with mean and std auc scores for each feature set
    """
    results = []
    
    for set_name, features in feature_sets.items():
        # filter to available features in X_train
        available_features = [f for f in features if f in X_train.columns]
        
        if len(available_features) == 0:
            print(f"skipping {set_name}: no features available")
            continue
            
        # create baseline model
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=random_state,
            n_jobs=-1,
            class_weight='balanced'
        )
        
        # cross-validation
        scores = cross_val_score(
            model, 
            X_train[available_features], 
            y_train,
            cv=cv,
            scoring='roc_auc',
            n_jobs=-1
        )
        
        results.append({
            'feature_set': set_name,
            'n_features': len(available_features),
            'mean_auc': scores.mean(),
            'std_auc': scores.std()
        })
        
        print(f"{set_name}: {scores.mean():.4f} ± {scores.std():.4f}")
    
    return pd.DataFrame(results).sort_values('mean_auc', ascending=False)

# execute comparison
feature_set_results = compare_feature_sets(X_train, y_train, feature_sets, cv=5)
feature_set_results

minimal_set: 0.6947 ± 0.0013
recommended_set: 0.7000 ± 0.0013
comprehensive_set: 0.7000 ± 0.0013
tier1_features: 0.6947 ± 0.0013
tier2_features: 0.5947 ± 0.0015
skipping tier3_features: no features available
tier4_features: 0.6656 ± 0.0038


,feature_set,n_features,mean_auc,std_auc
1,recommended_set,4,0.699953,0.001315
2,comprehensive_set,4,0.699953,0.001315
0,minimal_set,3,0.694675,0.001301
3,tier1_features,3,0.694675,0.001301
5,tier4_features,22,0.665602,0.003757
4,tier2_features,1,0.594736,0.001476


## Hyperparameter Optimisation
| Method | Mechanism | Advantages | Disadvantages | Use When |
|--------|-----------|------------|---------------|----------|
| **Grid Search** | exhaustive search over specified parameter grid | guaranteed to find best combination in grid | computationally expensive, curse of dimensionality | small parameter space, need comprehensive search |
| **Random Search** | random sampling from parameter distributions | more efficient than grid for large spaces | may miss optimal combination | large parameter space, limited compute budget |
| **Bayesian Optimisation** | builds probabilistic model of objective function | efficient for expensive evaluations | complex implementation, requires careful tuning | expensive model training, need sample efficiency |

### Random Forest Hyperparameter Tuning
Optimise random forest using randomised search for computational efficiency.

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform

def tune_random_forest(X_train: pd.DataFrame, y_train: pd.Series,
                       n_iter: int = 50,
                       cv: int = 5,
                       random_state: int = 2025) -> Dict:
    """
    hyperparameter tuning for random forest using randomised search
    
    parameter explanations:
    - n_estimators: number of trees in forest (more trees = better performance but slower)
    - max_depth: maximum tree depth (None = unlimited, higher = more complex, risk overfitting)
    - min_samples_split: minimum samples required to split node (higher = more conservative splits)
    - min_samples_leaf: minimum samples required at leaf node (higher = smoother decision boundaries)
    - max_features: number of features to consider for best split ('sqrt' = square root of total features)
    - bootstrap: whether to use bootstrap samples (True = sample with replacement)
    
    args:
        X_train: training features
        y_train: training target
        n_iter: number of parameter combinations to try
        cv: cross-validation folds
        random_state: random seed
        
    returns:
        dictionary with best parameters, best score, and fitted model
    """
    # define parameter distributions
    param_distributions = {
        'n_estimators': randint(100, 500),           # uniform int between 100-500
        'max_depth': [10, 20, 30, None],             # discrete choices
        'min_samples_split': randint(50, 200),       # uniform int between 50-200
        'min_samples_leaf': randint(25, 100),        # uniform int between 25-100
        'max_features': ['sqrt', 'log2', None],      # feature selection strategies
        'bootstrap': [True],                          # always use bootstrap
        'class_weight': ['balanced']                  # handle class imbalance
    }
    
    # base model
    rf = RandomForestClassifier(random_state=random_state, n_jobs=-1)
    
    # randomised search
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=random_state,
        verbose=2,
        return_train_score=True
    )
    
    print(f"starting randomised search with {n_iter} iterations...")
    random_search.fit(X_train, y_train)
    
    print(f"\nbest auc: {random_search.best_score_:.4f}")
    print(f"best parameters: {random_search.best_params_}")
    
    return {
        'best_params': random_search.best_params_,
        'best_score': random_search.best_score_,
        'best_model': random_search.best_estimator_,
        'cv_results': pd.DataFrame(random_search.cv_results_)
    }

# execute tuning
rf_tuning_results = tune_random_forest(X_train, y_train, n_iter=10, cv=5)

starting randomised search with 10 iterations...
Fitting 5 folds for each of 10 candidates, totalling 50 fits

best auc: 0.7191
best parameters: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 41, 'min_samples_split': 114, 'n_estimators': 473}


**Key Parameter Explanations**:

- **n_estimators**: Number of decision trees in the ensemble. More trees generally improve performance through better averaging but increase computational cost. Typical range: 100-500.

- **max_depth**: Maximum depth of each tree. Controls model complexity:
  - Shallow trees (5-10): prevent overfitting, faster training, may underfit
  - Medium trees (10-30): balanced approach for most problems
  - Deep trees (>30 or None): capture complex patterns, risk overfitting

- **min_samples_split**: Minimum samples required to split an internal node. Higher values:
  - Prevent overfitting by making splits more conservative
  - Create simpler trees with fewer nodes
  - Typical range: 2-200 depending on dataset size

- **min_samples_leaf**: Minimum samples required at leaf node. Higher values:
  - Smooth decision boundaries
  - Prevent overfitting to noise
  - Must be < min_samples_split

- **max_features**: Number of features considered for best split:
  - 'sqrt': square root of total features (good default for classification)
  - 'log2': log base 2 of total features (more conservative)
  - None: consider all features (higher variance between trees)

- **RandomizedSearchCV parameters**:
  - **n_iter**: number of parameter combinations to sample
  - **cv**: number of cross-validation folds
  - **verbose**: controls output verbosity (0=silent, 1=progress, 2=detailed)
  - **return_train_score**: whether to compute training scores (useful for diagnosing overfitting)

### Logistic Regression Hyperparameter Tuning
Optimise logistic regression with focus on regularisation and solver selection.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

def tune_logistic_regression(X_train: pd.DataFrame, y_train: pd.Series,
                             cv: int = 5,
                             random_state: int = 2025) -> Dict:
    """
    hyperparameter tuning for logistic regression using grid search
    
    parameter explanations:
    - C: inverse of regularisation strength (smaller = stronger regularisation)
    - penalty: regularisation type ('l1' = lasso, 'l2' = ridge, 'elasticnet' = combination)
    - solver: optimisation algorithm ('lbfgs' = limited-memory bfgs, 'saga' = stochastic average gradient)
    - max_iter: maximum iterations for convergence
    - class_weight: handling class imbalance
    
    args:
        X_train: training features (must be scaled)
        y_train: training target
        cv: cross-validation folds
        random_state: random seed
        
    returns:
        dictionary with tuning results
    """
    # parameter grid
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],        # regularisation strength
        'penalty': ['l2'],                           # l2 regularisation (ridge)
        'solver': ['lbfgs', 'saga'],                 # optimisation algorithms
        'max_iter': [5000],                          # sufficient for convergence
        'class_weight': ['balanced']                 # handle imbalance
    }
    
    # base model
    lr = LogisticRegression(random_state=random_state)
    
    # grid search
    grid_search = GridSearchCV(
        estimator=lr,
        param_grid=param_grid,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=2,
        return_train_score=True
    )
    
    print("starting grid search...")
    grid_search.fit(X_train, y_train)
    
    print(f"\nbest auc: {grid_search.best_score_:.4f}")
    print(f"best parameters: {grid_search.best_params_}")
    
    return {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_model': grid_search.best_estimator_,
        'cv_results': pd.DataFrame(grid_search.cv_results_)
    }

# note: ensure X_train is scaled before using logistic regression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_tuning_results = tune_logistic_regression(
    pd.DataFrame(X_train_scaled, columns=X_train.columns), 
    y_train, 
    cv=5
)

starting grid search...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

best auc: 0.7107
best parameters: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 5000, 'penalty': 'l2', 'solver': 'saga'}


**Key Parameter Explanations**:

- **C (Regularisation Parameter)**:
  - Inverse of regularisation strength (C = 1/λ)
  - Small C (e.g., 0.001): strong regularisation, simple model, may underfit
  - Large C (e.g., 100): weak regularisation, complex model, may overfit
  - Default C=1.0 is often reasonable starting point

- **penalty (Regularisation Type)**:
  - **'l1' (Lasso)**: sum of absolute weights, creates sparse models (some weights = 0)
  - **'l2' (Ridge)**: sum of squared weights, shrinks all weights but rarely to zero
  - **'elasticnet'**: combination of l1 and l2, requires additional l1_ratio parameter
  - Choice depends on feature selection needs and model interpretability requirements

- **solver (Optimisation Algorithm)**:
  - **'lbfgs'**: limited-memory bfgs, good for small datasets, only supports l2
  - **'saga'**: stochastic average gradient, good for large datasets, supports all penalties
  - **'liblinear'**: good for small datasets, supports l1 and l2
  - **'newton-cg'**: newton-conjugate gradient, good for multinomial problems

## Handling Class Imbalance
Apply Synthetic Minority Over-sampling Technique to balance classes.
**SMOTE (Synthetic Minority Over-sampling Technique) Explained**:

SMOTE creates synthetic examples of the minority class rather than simply duplicating existing examples. The algorithm:

1. **Select minority class sample**: Choose a random sample from minority class
2. **Find k-nearest neighbours**: Identify k nearest minority class neighbours (typically k=5)
3. **Generate synthetic sample**: 
   - Randomly select one of the k neighbours
   - Create new sample along the line segment connecting the original sample and selected neighbour
   - Formula: `synthetic = original + λ × (neighbour - original)` where λ ∈ [0, 1]
4. **Repeat**: Generate synthetic samples until desired balance achieved

In [19]:
from imblearn.over_sampling import SMOTE
from collections import Counter

def apply_smote(X_train: pd.DataFrame, y_train: pd.Series,
                sampling_strategy: float = 0.5,
                k_neighbors: int = 5,
                random_state: int = 2025) -> Tuple[pd.DataFrame, pd.Series]:
    """
    apply smote to handle class imbalance
    
    parameter explanations:
    - sampling_strategy: desired ratio of minority to majority class after resampling
      * float (e.g., 0.5): minority will be 50% of majority
      * 'auto': resample minority to match majority (1:1 ratio)
      * dict: specify exact counts for each class
    
    - k_neighbors: number of nearest neighbours for synthetic sample generation
      * typical: 5 (default)
      * smaller k: synthetic samples closer to original
      * larger k: more diverse synthetic samples, risk of overlap with majority class
    
    - random_state: ensures reproducible synthetic sample generation
    
    args:
        X_train: training features
        y_train: training target
        sampling_strategy: target ratio of minority to majority
        k_neighbors: neighbours for synthetic generation
        random_state: random seed
        
    returns:
        resampled X_train and y_train
    """
    print("original class distribution:")
    print(Counter(y_train))
    print(f"imbalance ratio: {(y_train==0).sum() / (y_train==1).sum():.2f}:1")
    
    # apply smote
    smote = SMOTE(
        sampling_strategy=sampling_strategy,
        k_neighbors=k_neighbors,
        random_state=random_state,
        
    )
    
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    print(f"\nresampled class distribution:")
    print(Counter(y_resampled))
    print(f"new imbalance ratio: {(y_resampled==0).sum() / (y_resampled==1).sum():.2f}:1")
    print(f"added {len(y_resampled) - len(y_train)} synthetic samples")
    
    return pd.DataFrame(X_resampled, columns=X_train.columns), pd.Series(y_resampled)

# apply smote
X_train_smote, y_train_smote = apply_smote(
    X_train, 
    y_train, 
    sampling_strategy=0.5,  # minority will be 50% of majority
    k_neighbors=5
)

original class distribution:
Counter({0: 166702, 1: 44908})
imbalance ratio: 3.71:1

resampled class distribution:
Counter({0: 166702, 1: 83351})
new imbalance ratio: 2.00:1
added 38443 synthetic samples


## Model Evaluation and Comparison
Systematically compare all tuned models on multiple metrics

In [23]:
# 1. Check what Random Forest is actually predicting
# The tuning object has the best model in .best_estimator_
rf_predictions = lr_tuning_results.best_estimator_.predict(X_test)
print("Unique RF predictions:", np.unique(rf_predictions, return_counts=True))
print("Actual test labels:", np.unique(y_test, return_counts=True))

# 2. Check the hyperparameters used
print("\nRF hyperparameters:", rf_tuned.get_params())

# 3. Check if it's predicting probabilities correctly
rf_proba = rf_tuned.predict_proba(X_test)
print("\nProbability distribution:")
print(f"Min: {rf_proba.min()}, Max: {rf_proba.max()}")
print(f"Sample probabilities:\n{rf_proba[:10]}")

AttributeError: 'dict' object has no attribute 'best_estimator_'

In [25]:
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

def comprehensive_model_evaluation(models_dict: Dict,
                                   X_test: pd.DataFrame,
                                   y_test: pd.Series,
                                   threshold: float = 0.5) -> pd.DataFrame:
    """
    evaluate multiple models on test set with comprehensive metrics
    
    metrics explained:
    - roc_auc: discriminative ability across all thresholds
    - precision: proportion of positive predictions that are correct
    - recall: proportion of actual positives correctly identified
    - f1: harmonic mean balancing precision and recall
    - specificity: proportion of actual negatives correctly identified
    - false positive rate: proportion of negatives incorrectly classified as positive
    
    args:
        models_dict: dictionary of fitted models
        X_test: test features
        y_test: test target
        threshold: classification threshold for binary predictions
        
    returns:
        comparison dataframe
    """
    results = []
    
    for name, model in models_dict.items():
        # predictions
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        y_pred = (y_pred_proba >= threshold).astype(int)
        
        # confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        # metrics
        results.append({
            'model': name,
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'specificity': tn / (tn + fp),
            'fpr': fp / (fp + tn),
            'threshold': threshold,
            'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
        })
    
    results_df = pd.DataFrame(results).sort_values('roc_auc', ascending=False)
    
    print("model comparison on test set:")
    print(results_df[['model', 'roc_auc', 'precision', 'recall', 'f1']].to_string(index=False))
    
    return results_df

# evaluate all tuned models
all_models = {
    'lr_tuned': lr_tuning_results['best_model'],
    'rf_tuned': rf_tuning_results['best_model'],

}

comparison_results = comprehensive_model_evaluation(
    all_models,
    X_test_scaled,  # use scaled features for models that need it
    y_test,
    threshold=0.5
)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


model comparison on test set:
   model  roc_auc  precision   recall       f1
lr_tuned 0.716112    0.34765 0.650213 0.453061
rf_tuned 0.597937    0.00000 0.000000 0.000000


/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Learning Curves for Diagnosing Issues

In [17]:
from sklearn.model_selection import learning_curve

def plot_learning_curves(model, X: pd.DataFrame, y: pd.Series,
                        cv: int = 5,
                        train_sizes: np.ndarray = np.linspace(0.1, 1.0, 10)) -> None:
    """
    generate learning curves to diagnose bias/variance
    
    interpretation:
    - high train score, low validation score: overfitting (high variance)
    - both scores low: underfitting (high bias)
    - both scores high and close: good fit
    - scores converging with more data: model will benefit from more data
    - large gap persists: model complexity issue, not data issue
    
    args:
        model: sklearn estimator
        X: features
        y: target
        cv: cross-validation folds
        train_sizes: fractions of training set to use
    """
    train_sizes, train_scores, val_scores = learning_curve(
        estimator=model,
        X=X,
        y=y,
        cv=cv,
        train_sizes=train_sizes,
        scoring='roc_auc',
        n_jobs=-1
    )
    
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, label='training score', marker='o')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    plt.plot(train_sizes, val_mean, label='validation score', marker='s')
    plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)
    
    plt.xlabel('training set size')
    plt.ylabel('roc auc score')
    plt.title('learning curves')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
    
    # diagnosis
    final_gap = train_mean[-1] - val_mean[-1]
    if final_gap > 0.1:
        print("diagnosis: overfitting detected (train >> validation)")
        print("recommendations: increase regularisation, reduce model complexity, or gather more data")
    elif val_mean[-1] < 0.7:
        print("diagnosis: underfitting detected (both scores low)")
        print("recommendations: increase model complexity, add features, or reduce regularisation")
    else:
        print("diagnosis: good fit (scores high and close)")